# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

from entity import Gene
import conf

# Settings

In [3]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "lincs" / "analyses"
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/analyses')

In [4]:
# OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
# display(OUTPUT_DIR)

# assert OUTPUT_DIR.exists()
# # OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [5]:
# INPUT_DATA_DIR = Path(
#     conf.RESULTS["DRUG_DISEASE_ANALYSES"],
#     "data",
# )
# display(INPUT_DATA_DIR)

In [6]:
# OUTPUT_PREDICTIONS_DIR = Path(
#     conf.RESULTS["DRUG_DISEASE_ANALYSES"], "predictions", "dotprod_neg"
# )
# display(OUTPUT_PREDICTIONS_DIR)

In [7]:
INPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "lincs" / "predictions"
# display(OUTPUT_DIR)
# OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
input_predictions_by_tissue_file = INPUT_DIR / "full_predictions_by_tissue-rank.h5"
display(input_predictions_by_tissue_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/predictions/full_predictions_by_tissue-rank.h5')

# Data loading

## S-MultiXcan projection

In [8]:
# input_file = Path(
#     conf.RESULTS["PROJECTIONS_DIR"],
#     "projection-smultixcan-efo_partial-mashr-zscores.pkl",
# ).resolve()
# display(input_file)

In [9]:
# smultixcan_proj = pd.read_pickle(input_file)

In [10]:
# smultixcan_proj.shape

In [11]:
# smultixcan_proj.head()

## S-MultiXcan

In [12]:
# # smultixcan_zscores = pd.read_pickle(conf.PHENOMEXCAN["SMULTIXCAN_MASHR_ZSCORES_FILE"])
# smultixcan_zscores = pd.read_pickle(
#     conf.PHENOMEXCAN["SMULTIXCAN_EFO_PARTIAL_MASHR_ZSCORES_FILE"]
# )

In [13]:
# smultixcan_zscores = smultixcan_zscores.rename(index=Gene.GENE_ID_TO_NAME_MAP)

In [14]:
# smultixcan_zscores = smultixcan_zscores[~smultixcan_zscores.index.duplicated()]

In [15]:
# smultixcan_zscores = smultixcan_zscores.dropna(how="any")

In [16]:
# smultixcan_zscores.shape

In [17]:
# smultixcan_zscores.head()

## PharmacotherapyDB: load gold standard

### Final

In [18]:
gold_standard = pd.read_pickle(
    Path(conf.RESULTS["DRUG_DISEASE_ANALYSES"], "gold_standard.pkl"),
)

In [19]:
gold_standard.shape

(998, 3)

In [20]:
gold_standard.head()

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


### Info

In [21]:
# TODO hardcoded
input_file = Path(
    conf.DATA_DIR, "hetionet/pharmacotherapydb-v1.0", "indications.tsv"
).resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases_data/base_orig/data/hetionet/pharmacotherapydb-v1.0/indications.tsv')

In [22]:
gold_standard_info = pd.read_csv(input_file, sep="\t")

In [23]:
gold_standard_info = gold_standard_info.rename(columns={"drug": "drug_name"})

In [24]:
gold_standard_info.shape

(1388, 7)

In [25]:
gold_standard_info.head()

,doid_id,drugbank_id,disease,drug_name,category,n_curators,n_resources
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,DM,2,1
1,DOID:10652,DB00674,Alzheimer's disease,Galantamine,DM,1,4
2,DOID:10652,DB01043,Alzheimer's disease,Memantine,DM,1,3
3,DOID:10652,DB00989,Alzheimer's disease,Rivastigmine,DM,1,3
4,DOID:10652,DB00245,Alzheimer's disease,Benzatropine,SYM,3,1


In [26]:
gold_standard_info = (
    gold_standard.set_index(["trait", "drug"])
    .join(
        gold_standard_info.rename(
            columns={"doid_id": "trait", "drugbank_id": "drug"}
        ).set_index(["trait", "drug"])
    )
    .reset_index()
)

In [27]:
gold_standard_info.shape

(998, 8)

In [28]:
gold_standard_info.head()

,trait,drug,true_class,disease,drug_name,category,n_curators,n_resources
0,DOID:10652,DB00843,1,Alzheimer's disease,Donepezil,DM,2,1
1,DOID:10652,DB00674,1,Alzheimer's disease,Galantamine,DM,1,4
2,DOID:10652,DB01043,1,Alzheimer's disease,Memantine,DM,1,3
3,DOID:10652,DB00989,1,Alzheimer's disease,Rivastigmine,DM,1,3
4,DOID:10652,DB00810,0,Alzheimer's disease,Biperiden,NOT,2,1


## LINCS data

In [29]:
input_file = Path(
    conf.RESULTS["DRUG_DISEASE_ANALYSES"], "lincs", "lincs-data.pkl"
).resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/lincs-data.pkl')

In [30]:
lincs_data = pd.read_pickle(input_file).T.rename(columns=Gene.GENE_ID_TO_NAME_MAP)

In [31]:
display(lincs_data.shape)

(1170, 7120)

In [32]:
display(lincs_data.head())

,ADA,CDH2,AKT3,MED6,ACOT8,ABI1,GNPDA1,CDH3,TANK,TOPORS-AS1,...,RBX1,CDC42,THOC1,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,PTBP3,CASP8AP2
perturbagen,,,,,,,,,,,,,,,,,,,,,
DB00014,-1.001,1.146,-0.693,-0.037,0.162,-0.380,0.649,-1.952,0.294,0.274,...,-0.592,0.178,0.270,-0.013,0.351,-0.128,0.274,-0.734,-0.048,0.036
DB00091,-1.835,-1.863,1.694,0.383,-0.899,0.628,-4.878,2.527,1.709,-0.981,...,-0.668,-2.795,-0.333,-0.027,0.578,6.926,-1.875,4.382,0.266,-4.995
DB00121,1.391,0.011,-0.804,0.269,0.105,-0.588,-1.899,0.306,-1.178,0.120,...,-0.962,0.450,-0.999,1.358,-1.476,0.423,-1.356,-1.897,-0.299,-0.732
DB00130,1.132,-1.020,-0.164,-0.997,-0.090,0.195,-2.341,0.494,-0.813,-1.140,...,-0.553,-0.528,0.308,0.534,-0.320,3.001,-0.414,0.134,0.147,2.158
DB00131,0.257,1.143,1.145,0.185,-1.291,-0.457,0.038,1.442,-1.692,0.593,...,-0.556,-0.600,0.092,-0.564,0.174,-1.203,-1.080,-0.575,-0.792,-0.095


In [33]:
lincs_data.index.is_unique

True

In [34]:
lincs_data.columns.is_unique

True

## LINCS projection

In [35]:
input_file = Path(
    conf.RESULTS["DRUG_DISEASE_ANALYSES"], "lincs", "lincs-projection.pkl"
).resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/lincs-projection.pkl')

In [36]:
lincs_projection = pd.read_pickle(input_file).T

In [37]:
display(lincs_projection.shape)

(1170, 987)

In [38]:
display(lincs_projection.head())

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
perturbagen,,,,,,,,,,,,,,,,,,,,,
DB00014,0.036115,0.012281,-0.005533,-0.004151,-0.015156,-0.010861,0.008181,-0.008148,0.004948,-0.009869,...,-0.003139,0.003546,-0.006242,0.002449,-0.004391,-0.004334,-0.019902,0.004292,-0.010378,0.002000
DB00091,-0.358610,0.009738,-0.059174,-0.032884,-0.005276,0.063369,-0.079522,-0.018980,-0.025782,0.021540,...,-0.019892,0.003610,-0.016165,0.008686,0.065335,-0.042051,0.065516,0.016261,0.056314,-0.051035
DB00121,0.091067,0.004650,0.013454,0.005934,0.025747,0.004324,-0.001865,-0.019613,0.051503,0.013032,...,0.001629,-0.012565,0.009613,-0.014404,-0.009188,0.006544,0.047466,-0.016220,-0.018844,0.034675
DB00130,0.008923,-0.006554,0.008906,-0.003991,0.000346,0.008212,0.001242,-0.012771,0.014221,0.008269,...,-0.026139,-0.002977,0.021969,0.001204,-0.012731,-0.000076,-0.029127,-0.022679,0.016179,-0.003029
DB00131,0.046469,0.021758,-0.004939,-0.028524,-0.013932,0.012231,-0.001325,0.003787,0.035189,-0.004377,...,0.017373,0.005304,-0.001649,0.004017,-0.000622,-0.006477,0.021663,0.000988,0.002038,-0.011747


## MultiPLIER Z

In [39]:
multiplier_z = pd.read_pickle(conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"])

In [40]:
multiplier_z.shape

(6750, 987)

In [41]:
multiplier_z.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


## Prediction results

### Full

In [42]:
output_file = Path(
    conf.RESULTS["DRUG_DISEASE_ANALYSES"],
    "lincs",
    "predictions",
    "predictions_results.pkl",
).resolve()
display(output_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/predictions/predictions_results.pkl')

In [43]:
predictions = pd.read_pickle(output_file)

In [44]:
predictions.shape

(316540, 8)

In [45]:
predictions.head()

,trait,drug,score,true_class,method,n_top_genes,data,tissue
0,DOID:0050741,DB00215,323.0,1,Gene-based,100.0,spredixcan-mashr-zscores-Pancreas-data,Pancreas
1,DOID:0050741,DB00704,230.0,1,Gene-based,100.0,spredixcan-mashr-zscores-Pancreas-data,Pancreas
2,DOID:0050741,DB00822,199.0,1,Gene-based,100.0,spredixcan-mashr-zscores-Pancreas-data,Pancreas
3,DOID:10283,DB00014,115.0,1,Gene-based,100.0,spredixcan-mashr-zscores-Pancreas-data,Pancreas
4,DOID:10283,DB00175,167.0,0,Gene-based,100.0,spredixcan-mashr-zscores-Pancreas-data,Pancreas


In [46]:
# def _reduce_mean(x):
#     return pd.Series(
#         {"score": x["score"].mean(), "true_class": x["true_class"].unique()[0]}
#     )

In [47]:
# predictions_by_tissue = (
#     predictions.groupby(["trait", "drug", "method", "tissue"])
#     .apply(_reduce_mean)
#     .dropna()
#     .sort_index()
#     .reset_index()
# )

In [48]:
# predictions_by_tissue.head()

### Aggregated

In [49]:
output_file = Path(
    conf.RESULTS["DRUG_DISEASE_ANALYSES"],
    "lincs",
    "predictions",
    "predictions_results_aggregated.pkl",
).resolve()
display(output_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/lincs/predictions/predictions_results_aggregated.pkl')

In [50]:
predictions_avg = pd.read_pickle(output_file)

In [51]:
predictions_avg.shape

(1292, 5)

In [52]:
predictions_avg.head()

,trait,drug,method,score,true_class
0,DOID:0050741,DB00215,Gene-based,359.0,1.0
1,DOID:0050741,DB00215,Module-based,437.8,1.0
2,DOID:0050741,DB00704,Gene-based,395.8,1.0
3,DOID:0050741,DB00704,Module-based,562.8,1.0
4,DOID:0050741,DB00822,Gene-based,556.8,1.0


### Merge

In [53]:
pharmadb_predictions = pd.merge(
    gold_standard_info,
    predictions_avg,
    #     left_on=["doid_id", "drugbank_id"],
    on=["trait", "drug"],
    how="inner",
)

In [54]:
pharmadb_predictions

,trait,drug,true_class_x,disease,drug_name,category,n_curators,n_resources,method,score,true_class_y
0,DOID:10652,DB00843,1,Alzheimer's disease,Donepezil,DM,2,1,Gene-based,311.6,1.0
1,DOID:10652,DB00843,1,Alzheimer's disease,Donepezil,DM,2,1,Module-based,475.2,1.0
2,DOID:10652,DB00674,1,Alzheimer's disease,Galantamine,DM,1,4,Gene-based,509.4,1.0
3,DOID:10652,DB00674,1,Alzheimer's disease,Galantamine,DM,1,4,Module-based,508.0,1.0
4,DOID:10652,DB01043,1,Alzheimer's disease,Memantine,DM,1,3,Gene-based,452.4,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1287,DOID:363,DB00917,0,uterine cancer,Dinoprostone,NOT,3,1,Module-based,476.0,0.0
1288,DOID:363,DB00783,0,uterine cancer,Estradiol,NOT,2,1,Gene-based,556.0,0.0
1289,DOID:363,DB00783,0,uterine cancer,Estradiol,NOT,2,1,Module-based,603.0,0.0
1290,DOID:12306,DB00553,1,vitiligo,Methoxsalen,DM,3,1,Gene-based,550.8,1.0


In [55]:
pharmadb_predictions = pharmadb_predictions[
    ["trait", "drug", "disease", "drug_name", "method", "score", "true_class_x"]
].rename(columns={"true_class_x": "true_class", "drug_x": "drug"})

In [56]:
display(pharmadb_predictions.shape)
assert pharmadb_predictions.shape[0] == predictions_avg.shape[0]

(1292, 7)

In [57]:
pharmadb_predictions.head()

,trait,drug,disease,drug_name,method,score,true_class
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,Gene-based,311.6,1
1,DOID:10652,DB00843,Alzheimer's disease,Donepezil,Module-based,475.2,1
2,DOID:10652,DB00674,Alzheimer's disease,Galantamine,Gene-based,509.4,1
3,DOID:10652,DB00674,Alzheimer's disease,Galantamine,Module-based,508.0,1
4,DOID:10652,DB01043,Alzheimer's disease,Memantine,Gene-based,452.4,1


In [58]:
pharmadb_predictions["trait"].unique().shape

(53,)

In [59]:
pharmadb_predictions["drug"].unique().shape

(322,)

### Standardize

In [60]:
data_stats = pharmadb_predictions.groupby("method")["score"].describe()
display(data_stats)

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Gene-based,646.0,517.657585,93.950154,87.3,458.95,535.4,591.75,646.0
Module-based,646.0,530.429721,73.472425,220.4,484.10,535.0,587.90,646.0


In [61]:
# Standardize scores by method
def _standardize(x):
    return (x["score"] - data_stats.loc[x["method"], "mean"]) / data_stats.loc[
        x["method"], "std"
    ]

In [62]:
pharmadb_predictions = pharmadb_predictions.assign(
    score_std=pharmadb_predictions.apply(_standardize, axis=1)
)

In [63]:
pharmadb_predictions

,trait,drug,disease,drug_name,method,score,true_class,score_std
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,Gene-based,311.6,1,-2.193265
1,DOID:10652,DB00843,Alzheimer's disease,Donepezil,Module-based,475.2,1,-0.751707
2,DOID:10652,DB00674,Alzheimer's disease,Galantamine,Gene-based,509.4,1,-0.087893
3,DOID:10652,DB00674,Alzheimer's disease,Galantamine,Module-based,508.0,1,-0.305281
4,DOID:10652,DB01043,Alzheimer's disease,Memantine,Gene-based,452.4,1,-0.694598
...,...,...,...,...,...,...,...,...
1287,DOID:363,DB00917,uterine cancer,Dinoprostone,Module-based,476.0,0,-0.740818
1288,DOID:363,DB00783,uterine cancer,Estradiol,Gene-based,556.0,0,0.408114
1289,DOID:363,DB00783,uterine cancer,Estradiol,Module-based,603.0,0,0.987721
1290,DOID:12306,DB00553,vitiligo,Methoxsalen,Gene-based,550.8,1,0.352766


### Testing

In [64]:
_tmp = pharmadb_predictions.groupby("method")[["score", "score_std"]].describe()
display(_tmp)

score                                                       \
              count        mean        std    min     25%    50%     75%   
method                                                                     
Gene-based    646.0  517.657585  93.950154   87.3  458.95  535.4  591.75   
Module-based  646.0  530.429721  73.472425  220.4  484.10  535.0  587.90   

                    score_std                                         \
                max     count          mean  std       min       25%   
method                                                                 
Gene-based    646.0     646.0 -5.609548e-16  1.0 -4.580701 -0.624880   
Module-based  646.0     646.0  2.034836e-16  1.0 -4.219675 -0.630573   

                                            
                   50%       75%       max  
method                                      
Gene-based    0.188849  0.788635  1.366069  
Module-based  0.062204  0.782202  1.572975

In [65]:
_tmp0 = pharmadb_predictions[(pharmadb_predictions["method"] == "Gene-based")][
    ["score", "score_std"]
]

In [66]:
assert all(_tmp0.corr() > 0.99999)

In [67]:
_tmp0 = pharmadb_predictions[(pharmadb_predictions["method"] == "Module-based")][
    ["score", "score_std"]
]

In [68]:
assert all(_tmp0.corr() > 0.99999)

# List diseases

In [69]:
pharmadb_predictions["disease"].unique()

array(["Alzheimer's disease", "Crohn's disease", "Parkinson's disease",
       'alcohol dependence', 'allergic rhinitis', 'anemia',
       'ankylosing spondylitis', 'asthma', 'atherosclerosis',
       'bone cancer', 'brain cancer', 'breast cancer', 'cervical cancer',
       'chronic kidney failure', 'chronic obstructive pulmonary disease',
       'coronary artery disease', 'epilepsy syndrome',
       'esophageal cancer', 'gestational diabetes', 'glaucoma', 'gout',
       'hematologic cancer', 'hypertension', 'hypothyroidism',
       'kidney cancer', 'liver cancer', 'lung cancer', 'malaria',
       'melanoma', 'migraine', 'multiple sclerosis', 'nephrolithiasis',
       'obesity', 'osteoarthritis', 'osteoporosis', 'pancreatic cancer',
       'pancreatitis', 'periodontitis', 'prostate cancer', 'psoriasis',
       'psoriatic arthritis', 'rheumatoid arthritis', 'skin cancer',
       'stomach cancer', 'systemic lupus erythematosus',
       'testicular cancer', 'thyroid cancer', 'type 1 diabe

# Looks for differences in scores of both methods

In [70]:
np.all(pharmadb_predictions.groupby(["trait", "drug"]).count() == 2)

True

In [71]:
def _compare(x):
    assert x.shape[0] == 2
    x_sign = np.sign(x["score_std"].values)
    x0 = x.iloc[0]["score_std"]
    x1 = x.iloc[1]["score_std"]

    return pd.Series(
        {"different_sign": x_sign[0] != x_sign[1], "score_difference": np.abs(x0 - x1)}
    )

In [72]:
pharmadb_predictions = pharmadb_predictions.set_index(["trait", "drug"]).join(
    pharmadb_predictions.groupby(["trait", "drug"]).apply(_compare)
)

In [73]:
pharmadb_predictions.head()

disease   drug_name        method  score  \
trait        drug                                                           
DOID:0050741 DB00215  alcohol dependence  Citalopram    Gene-based  359.0   
             DB00215  alcohol dependence  Citalopram  Module-based  437.8   
             DB00704  alcohol dependence  Naltrexone    Gene-based  395.8   
             DB00704  alcohol dependence  Naltrexone  Module-based  562.8   
             DB00822  alcohol dependence  Disulfiram    Gene-based  556.8   

                      true_class  score_std  different_sign  score_difference  
trait        drug                                                              
DOID:0050741 DB00215           1  -1.688742           False          0.428001  
             DB00215           1  -1.260741           False          0.428001  
             DB00704           1  -1.297045            True          1.737622  
             DB00704           1   0.440577            True          1.737622  
             DB00822           1   0.416630           False          0.508483

In [74]:
def find_differences(trait_name):
    with pd.option_context(
        "display.max_rows", None, "display.max_columns", None, "max_colwidth", None
    ):
        _tmp = pharmadb_predictions[
            (pharmadb_predictions["disease"] == trait_name)
            & (pharmadb_predictions["different_sign"])
        ].sort_values(
            ["score_difference", "drug_name", "method"], ascending=[False, False, False]
        )
        display(_tmp)

## any disease

In [75]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "max_colwidth", None
):
    _tmp = pharmadb_predictions[
        (pharmadb_predictions["different_sign"])
        & (~pharmadb_predictions["disease"].str.contains("cancer"))  # avoid cancer
    ].sort_values(
        ["score_difference", "drug_name", "method"], ascending=[False, False, False]
    )
    display(_tmp.head(50))

disease            drug_name  \
trait        drug                                                     
DOID:10763   DB01136              hypertension           Carvedilol   
             DB01136              hypertension           Carvedilol   
DOID:4481    DB00959         allergic rhinitis   Methylprednisolone   
             DB00959         allergic rhinitis   Methylprednisolone   
             DB01234         allergic rhinitis        Dexamethasone   
             DB01234         allergic rhinitis        Dexamethasone   
             DB00443         allergic rhinitis        Betamethasone   
             DB00443         allergic rhinitis        Betamethasone   
DOID:10763   DB00374              hypertension         Treprostinil   
             DB00374              hypertension         Treprostinil   
             DB00384              hypertension          Triamterene   
             DB00384              hypertension          Triamterene   
             DB00966              hypertension          Telmisartan   
             DB00966              hypertension          Telmisartan   
             DB01001              hypertension           Salbutamol   
             DB01001              hypertension           Salbutamol   
DOID:7147    DB00620    ankylosing spondylitis        Triamcinolone   
             DB00620    ankylosing spondylitis        Triamcinolone   
DOID:10763   DB00227              hypertension           Lovastatin   
             DB00227              hypertension           Lovastatin   
             DB00820              hypertension            Tadalafil   
             DB00820              hypertension            Tadalafil   
             DB00598              hypertension            Labetalol   
             DB00598              hypertension            Labetalol   
             DB00905              hypertension          Bimatoprost   
             DB00905              hypertension          Bimatoprost   
             DB00774              hypertension   Hydroflumethiazide   
             DB00774              hypertension   Hydroflumethiazide   
DOID:4481    DB00741         allergic rhinitis       Hydrocortisone   
             DB00741         allergic rhinitis       Hydrocortisone   
DOID:2841    DB00938                    asthma           Salmeterol   
             DB00938                    asthma           Salmeterol   
DOID:8893    DB01091                 psoriasis           Butenafine   
             DB01091                 psoriasis           Butenafine   
DOID:4481    DB00620         allergic rhinitis        Triamcinolone   
             DB00620         allergic rhinitis        Triamcinolone   
DOID:10763   DB00436              hypertension  Bendroflumethiazide   
             DB00436              hypertension  Bendroflumethiazide   
             DB00758              hypertension          Clopidogrel   
             DB00758              hypertension          Clopidogrel   
DOID:9352    DB01200  type 2 diabetes mellitus        Bromocriptine   
             DB01200  type 2 diabetes mellitus        Bromocriptine   
DOID:10763   DB01194              hypertension         Brinzolamide   
             DB01194              hypertension         Brinzolamide   
DOID:0050741 DB00704        alcohol dependence           Naltrexone   
             DB00704        alcohol dependence           Naltrexone   
DOID:13189   DB01394                      gout           Colchicine   
             DB01394                      gout           Colchicine   
DOID:2841    DB00535                    asthma             Cefdinir   
             DB00535                    asthma             Cefdinir   

                            method  score  true_class  score_std  \
trait        drug                                                  
DOID:10763   DB01136  Module-based  289.2           1  -3.283269   
             DB01136    Gene-based  551.6           1   0.361281   
DOID:4481    DB00959  Module-based  538.4           1   0.108480   
      

## coronary artery disease

In [76]:
pharmadb_predictions[
    (pharmadb_predictions["disease"] == "coronary artery disease")
    & (pharmadb_predictions["drug_name"] == "Niacin")
    #     & (pharmadb_predictions["different_sign"])
].sort_values(
    ["score_difference", "drug_name", "method"], ascending=[False, False, False]
)

disease drug_name        method  score  \
trait     drug                                                              
DOID:3393 DB00627  coronary artery disease    Niacin  Module-based  601.0   
          DB00627  coronary artery disease    Niacin    Gene-based  565.4   

                   true_class  score_std  different_sign  score_difference  
trait     drug                                                              
DOID:3393 DB00627           1   0.960500           False          0.452333  
          DB00627           1   0.508167           False          0.452333

In [77]:
find_differences("coronary artery disease")

disease        drug_name        method  \
trait     drug                                                              
DOID:3393 DB01197  coronary artery disease        Captopril  Module-based   
          DB01197  coronary artery disease        Captopril    Gene-based   
          DB00451  coronary artery disease    Levothyroxine  Module-based   
          DB00451  coronary artery disease    Levothyroxine    Gene-based   
          DB00509  coronary artery disease  Dextrothyroxine  Module-based   
          DB00509  coronary artery disease  Dextrothyroxine    Gene-based   
          DB00700  coronary artery disease       Eplerenone  Module-based   
          DB00700  coronary artery disease       Eplerenone    Gene-based   
          DB00790  coronary artery disease      Perindopril  Module-based   
          DB00790  coronary artery disease      Perindopril    Gene-based   
          DB01039  coronary artery disease      Fenofibrate  Module-based   
          DB01039  coronary artery disease      Fenofibrate    Gene-based   

                   score  true_class  score_std  different_sign  \
trait     drug                                                    
DOID:3393 DB01197  580.2           1   0.677401            True   
          DB01197  448.0           1  -0.741431            True   
          DB00451  483.5           0  -0.638739            True   
          DB00451  551.3           0   0.358088            True   
          DB00509  483.5           0  -0.638739            True   
          DB00509  551.3           0   0.358088            True   
          DB00700  538.6           1   0.111202            True   
          DB00700  437.2           1  -0.856386            True   
          DB00790  555.8           1   0.345303            True   
          DB00790  498.0           1  -0.209234            True   
          DB01039  533.6           1   0.043149            True   
          DB01039  497.4           1  -0.215621            True   

                   score_difference  
trait     drug                       
DOID:3393 DB01197          1.418832  
          DB01197          1.418832  
          DB00451          0.996827  
          DB00451          0.996827  
          DB00509          0.996827  
          DB00509          0.996827  
          DB00700          0.967588  
          DB00700          0.967588  
          DB00790          0.554538  
          DB00790          0.554538  
          DB01039          0.258770  
          DB01039          0.258770

## atherosclerosis

In [78]:
pharmadb_predictions[
    (pharmadb_predictions["disease"] == "atherosclerosis")
    & (pharmadb_predictions["drug_name"] == "Niacin")
    #     & (pharmadb_predictions["different_sign"])
].sort_values(
    ["score_difference", "drug_name", "method"], ascending=[False, False, False]
)

disease drug_name        method  score  true_class  \
trait     drug                                                                  
DOID:1936 DB00627  atherosclerosis    Niacin  Module-based  568.6           1   
          DB00627  atherosclerosis    Niacin    Gene-based  516.6           1   

                   score_std  different_sign  score_difference  
trait     drug                                                  
DOID:1936 DB00627   0.519518            True          0.530775  
          DB00627  -0.011257            True          0.530775

In [79]:
find_differences("atherosclerosis")

disease    drug_name        method  score  \
trait     drug                                                         
DOID:1936 DB00227  atherosclerosis   Lovastatin  Module-based  439.0   
          DB00227  atherosclerosis   Lovastatin    Gene-based  553.8   
          DB00175  atherosclerosis  Pravastatin  Module-based  470.0   
          DB00175  atherosclerosis  Pravastatin    Gene-based  534.4   
          DB00758  atherosclerosis  Clopidogrel  Module-based  503.2   
          DB00758  atherosclerosis  Clopidogrel    Gene-based  569.4   
          DB00627  atherosclerosis       Niacin  Module-based  568.6   
          DB00627  atherosclerosis       Niacin    Gene-based  516.6   

                   true_class  score_std  different_sign  score_difference  
trait     drug                                                              
DOID:1936 DB00227           1  -1.244409            True          1.629106  
          DB00227           1   0.384698            True          1.629106  
          DB00175           1  -0.822482            True          1.000687  
          DB00175           1   0.178205            True          1.000687  
          DB00758           0  -0.370611            True          0.921355  
          DB00758           0   0.550743            True          0.921355  
          DB00627           1   0.519518            True          0.530775  
          DB00627           1  -0.011257            True          0.530775

# Niacin and Atherosclerosis/CAD

In [80]:
from entity import Trait

In [81]:
Trait.get_traits_from_efo("atherosclerosis")

[I70-Diagnoses_main_ICD10_I70_Atherosclerosis]

In [82]:
d = Trait.get_trait(full_code="I70-Diagnoses_main_ICD10_I70_Atherosclerosis")

In [83]:
d.n, d.n_cases

(361194, 566.0)

In [84]:
d.get_do_info()

EfoInfo(id=['DOID:1936'], label=None)

In [85]:
Trait.get_traits_from_efo("coronary artery disease")

[I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease,
 CARDIoGRAM_C4D_CAD_ADDITIVE]

In [86]:
# d = Trait.get_trait(full_code="CARDIoGRAM_C4D_CAD_ADDITIVE")
d = Trait.get_trait(
    full_code="I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease"
)

In [87]:
d.n, d.n_cases

(361194, 12769.0)

In [88]:
d.get_do_info()

EfoInfo(id=['DOID:3393'], label=None)

In [89]:
Trait.get_traits_from_efo("hypertension")

[20002_1065-Noncancer_illness_code_selfreported_hypertension]

In [90]:
d = Trait.get_trait(
    full_code="20002_1065-Noncancer_illness_code_selfreported_hypertension"
)

In [91]:
d.n, d.n_cases

(361141, 93560.0)

In [92]:
d.get_do_info()

EfoInfo(id=['DOID:10763'], label=None)

In [93]:
Trait.get_traits_from_efo("myocardial infarction")

[20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction]

In [94]:
d = Trait.get_trait(
    full_code="20002_1473-Noncancer_illness_code_selfreported_high_cholesterol"
)

In [95]:
d.n, d.n_cases

(361141, 43957.0)

In [96]:
d.get_do_info()

In [97]:
# _doid = "DOID:1936"
_phenomexcan_traits = [
    "I70-Diagnoses_main_ICD10_I70_Atherosclerosis",
    "CARDIoGRAM_C4D_CAD_ADDITIVE",
    "I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease",
    "20002_1065-Noncancer_illness_code_selfreported_hypertension",
    "20002_1473-Noncancer_illness_code_selfreported_high_cholesterol",
    # others
    "6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above",
    "6150_4-Vascularheart_problems_diagnosed_by_doctor_High_blood_pressure",
    # lipids
    "MAGNETIC_HDL.C",
    "MAGNETIC_LDL.C",
    "MAGNETIC_IDL.TG",
    "MAGNETIC_CH2.DB.ratio",
    #
    "6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack",
    "I9_CHD-Major_coronary_heart_disease_event",
    "I9_CORATHER-Coronary_atherosclerosis",
    "I9_IHD-Ischaemic_heart_disease_wide_definition",
    "I9_MI-Myocardial_infarction",
    "I9_MI_STRICT-Myocardial_infarction_strict",
    "I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction",
    "20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction",
]

# _doid = "DOID:3393"
# _phenomexcan_traits = [
#     "I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease",
#     "CARDIoGRAM_C4D_CAD_ADDITIVE"
# ]

_drug_id = "DB00627"
_drug_name = "Niacin"

In [98]:
pharmadb_predictions[pharmadb_predictions["drug_name"] == _drug_name].sort_values(
    ["disease", "method"]
)

disease drug_name        method  score  \
trait     drug                                                              
DOID:1936 DB00627          atherosclerosis    Niacin    Gene-based  516.6   
          DB00627          atherosclerosis    Niacin  Module-based  568.6   
DOID:3393 DB00627  coronary artery disease    Niacin    Gene-based  565.4   
          DB00627  coronary artery disease    Niacin  Module-based  601.0   
DOID:4989 DB00627             pancreatitis    Niacin    Gene-based  394.8   
          DB00627             pancreatitis    Niacin  Module-based  518.8   

                   true_class  score_std  different_sign  score_difference  
trait     drug                                                              
DOID:1936 DB00627           1  -0.011257            True          0.530775  
          DB00627           1   0.519518            True          0.530775  
DOID:3393 DB00627           1   0.508167           False          0.452333  
          DB00627           1   0.960500           False          0.452333  
DOID:4989 DB00627           0  -1.307689           False          1.149402  
          DB00627           0  -0.158287           False          1.149402

## Get best tissue results for traits

In [99]:
drugs_tissue_df = {}

with pd.HDFStore(input_predictions_by_tissue_file, mode="r") as store:
    for tk in store.keys():
        df = store[tk][_drug_id]

        drugs_tissue_df[tk[1:]] = df
#         tissue_df_flatten = tissue_df.values.flatten()
#         tissue_df = (tissue_df - tissue_df_flatten.mean()) / tissue_df_flatten.std()

#         tissue_df = tissue_df.loc[_phenomexcan_traits, _drug_id]

In [100]:
drug_df = pd.DataFrame(drugs_tissue_df).max(1)

In [101]:
drug_df.shape

(4091,)

In [102]:
drug_df.head()

I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities                       4240339
I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection        4231713
G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies                4573539
2395_4-Hairbalding_pattern_Pattern_4                               3979892
20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet    4452501
dtype: int64

In [103]:
drug_df.describe()

count    4.091000e+03
mean     4.240773e+06
std      3.346693e+05
min      2.317380e+06
25%      4.064316e+06
50%      4.323137e+06
75%      4.488742e+06
max      4.760801e+06
dtype: float64

In [104]:
traits_tissue_df = {}

with pd.HDFStore(input_predictions_by_tissue_file, mode="r") as store:
    for tk in store.keys():
        df = store[tk].loc[_phenomexcan_traits, _drug_id]

        traits_tissue_df[tk[1:]] = df
#         tissue_df_flatten = tissue_df.values.flatten()
#         tissue_df = (tissue_df - tissue_df_flatten.mean()) / tissue_df_flatten.std()

#         tissue_df = tissue_df.loc[_phenomexcan_traits, _drug_id]

In [105]:
traits_best_tissues_df = pd.DataFrame(traits_tissue_df).idxmax(1)

In [106]:
traits_best_tissues_df

I70-Diagnoses_main_ICD10_I70_Atherosclerosis                                                   Cells_Cultured_fibroblasts
CARDIoGRAM_C4D_CAD_ADDITIVE                                                                              Colon_Transverse
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease                                  Brain_Putamen_basal_ganglia
20002_1065-Noncancer_illness_code_selfreported_hypertension                                   Brain_Putamen_basal_ganglia
20002_1473-Noncancer_illness_code_selfreported_high_cholesterol                                            Brain_Amygdala
6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above                                               Liver
6150_4-Vascularheart_problems_diagnosed_by_doctor_High_blood_pressure                                             Stomach
MAGNETIC_HDL.C                                                                                     Heart_Atrial_Appendage
MAGNETIC_LDL.C          

In [107]:
traits_df = pd.DataFrame(traits_tissue_df).max(1)

In [108]:
traits_df

I70-Diagnoses_main_ICD10_I70_Atherosclerosis                                        4574511
CARDIoGRAM_C4D_CAD_ADDITIVE                                                         4598049
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease                        4687680
20002_1065-Noncancer_illness_code_selfreported_hypertension                         4441478
20002_1473-Noncancer_illness_code_selfreported_high_cholesterol                     4549772
6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above               4539449
6150_4-Vascularheart_problems_diagnosed_by_doctor_High_blood_pressure               4587815
MAGNETIC_HDL.C                                                                      4535385
MAGNETIC_LDL.C                                                                      4111780
MAGNETIC_IDL.TG                                                                     4028127
MAGNETIC_CH2.DB.ratio                                                           

In [109]:
traits_df.shape

(19,)

In [110]:
traits_df.sort_values()

MAGNETIC_CH2.DB.ratio                                                               3884220
20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction    3996589
6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack                      4014872
MAGNETIC_IDL.TG                                                                     4028127
MAGNETIC_LDL.C                                                                      4111780
I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction                            4242830
I9_MI-Myocardial_infarction                                                         4417032
I9_MI_STRICT-Myocardial_infarction_strict                                           4417032
20002_1065-Noncancer_illness_code_selfreported_hypertension                         4441478
I9_CHD-Major_coronary_heart_disease_event                                           4443732
MAGNETIC_HDL.C                                                                  

In [111]:
traits_drug_df = (traits_df - drug_df.mean()) / drug_df.std()

In [112]:
traits_drug_df.sort_values()

MAGNETIC_CH2.DB.ratio                                                              -1.065389
20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction   -0.729627
6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack                     -0.674997
MAGNETIC_IDL.TG                                                                    -0.635391
MAGNETIC_LDL.C                                                                     -0.385434
I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction                            0.006147
I9_MI-Myocardial_infarction                                                         0.526667
I9_MI_STRICT-Myocardial_infarction_strict                                           0.526667
20002_1065-Noncancer_illness_code_selfreported_hypertension                         0.599712
I9_CHD-Major_coronary_heart_disease_event                                           0.606447
MAGNETIC_HDL.C                                                        

In [113]:
d = Trait.get_trait(full_code="I9_IHD-Ischaemic_heart_disease_wide_definition")

In [114]:
d.n, d.n_cases

(361194, 20857.0)

In [115]:
_tmp_df = (drug_df - drug_df.mean()) / drug_df.std()

In [116]:
_tmp_df.describe().apply(str)

count                     4091.0
mean     -1.2991590475398556e-15
std           1.0000000000000002
min          -5.7471445571517945
25%          -0.5272574381504886
50%           0.2461059844517343
75%           0.7409392332288909
max           1.5538566267480678
dtype: object

In [117]:
_tmp_df.quantile([0.80, 0.85, 0.90, 0.95])

0.80    0.834881
0.85    0.938876
0.90    1.036803
0.95    1.185432
dtype: float64

In [118]:
def find_best_tissue(trait_id):
    return traits_best_tissues_df.loc[trait_id]

In [119]:
_tmp_res = find_best_tissue("I9_IHD-Ischaemic_heart_disease_wide_definition")
display(_tmp_res)

'Brain_Cerebellar_Hemisphere'

## Gene module-based - LVs driving association

In [120]:
# available_doids = set(predictions_by_tissue["trait"].unique())
traits_lv_data = []

for trait in _phenomexcan_traits:
    #     t = Trait.get_trait(full_code=trait)

    #     t_doid = t.get_do_info().id
    #     # select available doid
    #     t_doid = [x for x in t_doid if x in available_doids]
    #     assert len(t_doid) == 1
    #     t_doid = t_doid[0]
    #     display(t_doid)

    best_module_tissue = find_best_tissue(trait)
    display(best_module_tissue)

    best_module_tissue_data = pd.read_pickle(
        conf.RESULTS["DRUG_DISEASE_ANALYSES"]
        / "spredixcan"
        / "proj"
        / f"spredixcan-mashr-zscores-{best_module_tissue}-projection.pkl"
    )[trait]

    traits_lv_data.append(best_module_tissue_data)

'Cells_Cultured_fibroblasts'

'Colon_Transverse'

'Brain_Putamen_basal_ganglia'

'Brain_Putamen_basal_ganglia'

'Brain_Amygdala'

...

'Brain_Cerebellar_Hemisphere'

'Brain_Nucleus_accumbens_basal_ganglia'

'Brain_Nucleus_accumbens_basal_ganglia'

'Brain_Cerebellar_Hemisphere'

'Esophagus_Mucosa'

In [121]:
module_tissue_data = pd.DataFrame(traits_lv_data).T

In [122]:
module_tissue_data.shape

(987, 19)

In [123]:
module_tissue_data.head()

,I70-Diagnoses_main_ICD10_I70_Atherosclerosis,CARDIoGRAM_C4D_CAD_ADDITIVE,I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease,20002_1065-Noncancer_illness_code_selfreported_hypertension,20002_1473-Noncancer_illness_code_selfreported_high_cholesterol,6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above,6150_4-Vascularheart_problems_diagnosed_by_doctor_High_blood_pressure,MAGNETIC_HDL.C,MAGNETIC_LDL.C,MAGNETIC_IDL.TG,MAGNETIC_CH2.DB.ratio,6150_1-Vascularheart_problems_diagnosed_by_doctor_Heart_attack,I9_CHD-Major_coronary_heart_disease_event,I9_CORATHER-Coronary_atherosclerosis,I9_IHD-Ischaemic_heart_disease_wide_definition,I9_MI-Myocardial_infarction,I9_MI_STRICT-Myocardial_infarction_strict,I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction,20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction
LV1,0.005720,-0.002536,0.003223,0.133016,0.027183,-0.009975,0.076686,0.004675,0.043670,0.029299,-0.012070,0.032704,-0.050731,-0.041645,-0.034174,0.022980,0.022980,-0.028532,0.008422
LV2,-0.002927,-0.040910,0.009556,0.000760,0.031170,-0.000601,0.015566,-0.003369,-0.001206,0.003167,-0.011635,0.002790,-0.058849,-0.053181,-0.025907,0.009914,0.009914,-0.044439,-0.054301
LV3,0.002734,0.033692,-0.005872,0.018757,-0.015628,0.029873,-0.055076,0.008938,-0.017906,-0.010961,-0.048228,-0.026598,-0.034387,-0.018676,-0.020514,-0.001761,-0.001761,-0.027963,0.003448
LV4,0.015454,-0.017444,0.004483,-0.040484,0.000231,-0.073387,0.044005,-0.015783,-0.004416,0.018076,0.020984,-0.063318,-0.040422,0.022468,-0.046343,-0.009688,-0.009688,-0.033462,-0.005588
LV5,0.027796,0.016727,-0.009114,0.001534,-0.037452,0.032017,-0.033671,-0.012350,-0.003539,-0.044317,-0.007597,-0.010769,-0.020286,-0.030727,-0.015489,0.024017,0.024017,-0.009718,-0.009654


In [124]:
drug_data = lincs_projection.loc[_drug_id]

In [125]:
drug_data

LV1     -0.032338
LV2      0.007960
LV3      0.009136
LV4     -0.005717
LV5     -0.023626
           ...   
LV983    0.005209
LV984    0.021905
LV985    0.013012
LV986   -0.014168
LV987    0.012947
Name: DB00627, Length: 987, dtype: float64

In [126]:
_tmp = (-1.0 * drug_data.dot(module_tissue_data)).sort_values(ascending=False)
display(_tmp)

I9_IHD-Ischaemic_heart_disease_wide_definition                                      0.041041
6150_100-Vascularheart_problems_diagnosed_by_doctor_None_of_the_above               0.033455
I9_MI_STRICT-Myocardial_infarction_strict                                           0.030205
I9_MI-Myocardial_infarction                                                         0.030205
I9_CORATHER-Coronary_atherosclerosis                                                0.025396
I70-Diagnoses_main_ICD10_I70_Atherosclerosis                                        0.023000
I9_CHD-Major_coronary_heart_disease_event                                           0.021127
I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease                        0.021002
6150_4-Vascularheart_problems_diagnosed_by_doctor_High_blood_pressure               0.019493
20002_1473-Noncancer_illness_code_selfreported_high_cholesterol                     0.018520
I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction              

In [127]:
predictions_avg[
    predictions_avg["trait"].isin(("DOID:1936", "DOID:3393"))
    & (predictions_avg["drug"] == _drug_id)
]

,trait,drug,method,score,true_class
588,DOID:1936,DB00627,Gene-based,516.6,1.0
589,DOID:1936,DB00627,Module-based,568.6,1.0
886,DOID:3393,DB00627,Gene-based,565.4,1.0
887,DOID:3393,DB00627,Module-based,601.0,1.0


In [128]:
drug_trait_predictions = pd.DataFrame(
    drug_data.to_frame().values * module_tissue_data.values,
    columns=module_tissue_data.columns.copy(),
    index=drug_data.index.copy(),
)

In [129]:
_lvs_sel = ["LV246", "LV847", "LV136", "LV931", "LV116"]

In [130]:
drug_data.quantile([0.05, 0.20, 0.80, 0.95])

0.05   -0.027790
0.20   -0.011224
0.80    0.011868
0.95    0.023996
Name: DB00627, dtype: float64

In [131]:
drug_data.loc[_lvs_sel]

LV246   -0.042009
LV847    0.002689
LV136   -0.018860
LV931   -0.069832
LV116    0.097558
Name: DB00627, dtype: float64

In [150]:
drug_trait_predictions.loc[:, "I9_MI-Myocardial_infarction"].sort_values(
    ascending=True
).head(20)

LV881   -0.004827
LV841   -0.002385
LV116   -0.002296
LV227   -0.002247
LV879   -0.002102
LV97    -0.002069
LV272   -0.001990
LV582   -0.001665
LV583   -0.001586
LV142   -0.001553
LV707   -0.001497
LV684   -0.001424
LV74    -0.001370
LV90    -0.001364
LV965   -0.001347
LV202   -0.001337
LV246   -0.001277
LV132   -0.001223
LV479   -0.001219
LV507   -0.001200
Name: I9_MI-Myocardial_infarction, dtype: float64

In [132]:
for _trait in _phenomexcan_traits:
    #     display(_trait)
    _tmp = drug_trait_predictions.loc[_lvs_sel, _trait].sort_values(ascending=True)
    display(_tmp)

    d = Trait.get_trait(full_code=_trait)
    display((d.n, d.n_cases))

    print()

LV116   -0.006180
LV931   -0.004843
LV246   -0.001253
LV136    0.000022
LV847    0.000025
Name: I70-Diagnoses_main_ICD10_I70_Atherosclerosis, dtype: float64

(361194, 566.0)

LV136   -0.001332
LV116   -0.001011
LV847   -0.000037
LV931    0.000283
LV246    0.001622
Name: CARDIoGRAM_C4D_CAD_ADDITIVE, dtype: float64

(184305, 60801.0)

LV116   -0.005052
LV246   -0.000686
LV136   -0.000493
LV847   -0.000034
LV931    0.001128
Name: I25-Diagnoses_main_ICD10_I25_Chronic_ischaemic_heart_disease, dtype: float64

...

(361194, 7018.0)

LV116   -0.004386
LV246   -0.001440
LV931   -0.000324
LV847   -0.000027
LV136    0.000377
Name: I21-Diagnoses_main_ICD10_I21_Acute_myocardial_infarction, dtype: float64

(361194, 5948.0)

LV116   -0.003414
LV136   -0.000465
LV847    0.000015
LV246    0.000467
LV931    0.001656
Name: 20002_1075-Noncancer_illness_code_selfreported_heart_attackmyocardial_infarction, dtype: float64

(361141, 8239.0)